Bound state trajectories are available on Zenodo. These files were uploaded in notebook form but were usually run as python scripts inside the folder corresponding to each system
These scripts are just used to extract files which are later analysed to obtain relevant info.

In [ ]:

import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
import matplotlib.pyplot as plt
from MDAnalysis.analysis.distances import distance_array as dist_array
from MDAnalysis.lib.distances import calc_angles as ca
u = mda.Universe("topology.tpr", "selected_frames.xtc")
nucl_donor_query="(resname G and name N1 N2 O2' O3' O5') or (resname A and name N6 O2' O3' O5') or (resname U and name N3 O2' O3' O5') or (resname C and name N4 O2' O3' O5')"
nucl_acceptor_query="(name O1P O2P O3' O5' O2' O4') or (resname G and name O6 N1 N2 N7 N3 N9) or (resname A and name N6 N7 N1 N3 N9) or (resname U and name N3 O4 O2 N1) or (resname C and name N4 O2 N3 N1)"

#O5' and O3' are donors and acceptors when terminal, only acceptors when terminal, so you remove them manually from the donor list when not terminal
donors=u.select_atoms(nucl_donor_query)
addquery_donor=" and not bynum "
for donor in donors:
    fine=0
    for batom in donor.bonded_atoms:
        if batom.element=="H":
            fine=1
    if fine==0:
        addquery_donor+=str(donor.id+1)+" "
if not addquery_donor==" and not bynum ": 
    nucl_donor_query+=addquery_donor
    print(nucl_donor_query)
vdw={}
vdw["C"]=1.7
vdw["N"]=1.55
vdw["O"]=1.52
vdw["P"]=1.8
contacts=[]
hbonds = HBA(universe=u,update_selections=False,acceptors_sel="resname PC and name O*",donors_sel=nucl_donor_query,d_a_cutoff=3.5,d_h_a_angle_cutoff=130)
hbonds.run()
np.savetxt("hbonds_traj.dat",hbonds.results['hbonds'])
np.savetxt("hbonds_count.dat",hbonds.count_by_time())
for i_ts,ts in enumerate(u.trajectory):
    rna=u.select_atoms("nucleic and not name H*")
    memb=u.select_atoms("resname PC PA and not name H* and around 5 (nucleic and not name H*)")
    #You preselect atoms which are close to RNA since MDAnalysis does that fast.
    distances=dist_array(rna.positions,memb.positions,box=u.dimensions)
    contact=0
    for i_rna, rna_atom in enumerate(rna):
        for j_memb,memb_atom in enumerate(memb):
            threshold=vdw[rna_atom.element]+vdw[memb_atom.element]+0.6
            if distances[i_rna,j_memb]<=threshold:
                contact+=1
    contacts.append(contact)
contacts=np.asarray(contacts)
contacts=contacts-hbonds.count_by_time() #Hbonds are subtracted from the contacts, to get the number of contacts that are not hydrogen bonds
np.savetxt("vdw_rna_memb.dat",contacts)
#Since water can, of course, act both as a donor and an acceptor we perform two separate analyses. We then sum the results in later analyses
hbonds_wat_acceptor = HBA(universe=u,update_selections=False,acceptors_sel="resname TP3 WAT and name O*",donors_sel=nucl_donor_query,d_a_cutoff=3.5,d_h_a_angle_cutoff=130)
hbonds_wat_acceptor.run()
np.savetxt("hbonds_wat_acceptor_traj.dat",hbonds_wat_acceptor.results['hbonds'])
np.savetxt("hbonds_wat_acceptor_count.dat",hbonds_wat_acceptor.count_by_time())
hbonds_wat_donor = HBA(universe=u,update_selections=False,acceptors_sel=nucl_acceptor_query,donors_sel="resname TP3 WAT and name O*",d_a_cutoff=3.5, d_h_a_angle_cutoff=130)
hbonds_wat_donor.run()
np.savetxt("hbonds_wat_donor_traj.dat",hbonds_wat_donor.results['hbonds'])
np.savetxt("hbonds_wat_donor_count.dat",hbonds_wat_donor.count_by_time())


## vdW contacts per residue: relevant for strand and quadruplex

In [ ]:
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
import matplotlib.pyplot as plt
from MDAnalysis.analysis.distances import distance_array as dist_array
from MDAnalysis.lib.distances import calc_angles as ca
u = mda.Universe("topology.tpr", "selected_dframes.xtc")
vdw={}
vdw["C"]=1.7
vdw["N"]=1.55
vdw["O"]=1.52
vdw["P"]=1.8
res_contacts=[]
res_contacts_list=[]
for i_ts,ts in enumerate(u.trajectory):
    rna=u.select_atoms("nucleic and not name H*")
    memb=u.select_atoms("resname PC PA and not name H* and around 5 (nucleic and not name H*)")
    #You preselect atoms which are close to RNA since MDAnalysis does that fast.
    distances=dist_array(rna.positions,memb.positions,box=u.dimensions)
    residue_cont=np.zeros(19)
    for i_rna, rna_atom in enumerate(rna):
        for j_memb,memb_atom in enumerate(memb):
            threshold=vdw[rna_atom.element]+vdw[memb_atom.element]+0.6
            if distances[i_rna,j_memb]<=threshold:
                residue_cont[rna_atom.residue.resid-1]+=1
    res_contacts.append(np.sum(residue_cont))
    res_contacts_list.append(residue_cont)

res_contacts=np.asarray(res_contacts)
res_contacts_list=np.asarray(res_contacts_list)
np.savetxt("res_contacts.dat",res_contacts)
np.savetxt("res_contacts_list.dat",res_contacts_list)